# Section 1 - Scrape Web Page

Import the libaries and then Scrape the HTML table to a dataframe

In [56]:
import lxml.html as lh
import pandas as pd
import requests
import numpy as np

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
scrape_source = pd.read_html(url)

df_temp=scrape_source[0]

df_temp.sort_values(by='Postal Code',inplace=True)


Now clean the dataframe to remove unwanted values, rename columns and concatenate duplicate instances of the PostalCode values

In [20]:
df_subs = df_temp[["Postal Code","Borough","Neighbourhood"]]

df_subs.rename(columns ={"Postal Code":"PostalCode"},inplace=True)

df_subs.drop(df_subs.loc[df_subs['Borough']=="Not assigned"].index, inplace=True)

df_subs=df_subs.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

Make sure there are no instances of Not Assigned in the Neighbourhood col

In [21]:
df_subs[df_subs.Neighbourhood == "Not assigned"].shape[0]

0

Report the count of rows in the final table

In [22]:
df_subs.shape

(103, 3)

Show the first 5 rows of the final table

In [23]:
df_subs.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Section 2 - Merge Geospatial Data

In [26]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')

In [143]:
df_geo.rename(columns ={"Postal Code":"PostalCode"},inplace=True)

In [28]:
df_subs_geo=pd.merge(df_geo, df_subs, on="PostalCode")

# Section 3 - Cluster and generate Map

In [142]:
#Full install as Watson Studio doesnt like the other method
!pip install folium

from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [94]:
#Create the array to filter only subs with Toronto
df_tor=['Central Toronto', 'Downtown Toronto', 'East Toronto', 'West Toronto']
df_tor_only = df_subs_geo[df_subs_geo['Borough'].isin(df_tor)].reset_index(drop=True)

df_tor_only.head()

(39, 5)


,PostalCode,Latitude,Longitude,Borough,Neighbourhood
0,M4E,43.676357,-79.293031,East Toronto,The Beaches
1,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
2,M4L,43.668999,-79.315572,East Toronto,"India Bazaar, The Beaches West"
3,M4M,43.659526,-79.340923,East Toronto,Studio District
4,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park


In [97]:
#build the initial map to make sure that the subs are plotted correctly

map_tor = folium.Map(location=[43.653226, -79.383184], zoom_start=10)
for lat, lng, borough, neighbourhood in zip(df_tor_only['Latitude'], df_tor_only['Longitude'], df_tor_only['Borough'], df_tor_only['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_tor)  
    
map_tor

In [101]:
#Build out the data source using Foursquare on the suburb data we have

CLIENT_ID = 'M0NK41BIWIW2KGFFGTRUVC2TG1R1PCMPK55TQRHUDXDDBZG1' 
CLIENT_SECRET = 'GXU1ZJENSAXGDHYMPOWZGCGMZ5NWUFXQRYRBWZ52QIUUBRXI' 
VERSION = '20180605' 
LIMIT = 100 
venues = []
radius = 750


for lat, long, post, borough, neighbourhood in zip(df_tor_only['Latitude'], df_tor_only['Longitude'], df_tor_only['PostalCode'], df_tor_only['Borough'], df_tor_only['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [141]:
#Create a dataframe to hold the data source generated from Foursqure
df_search_data = pd.DataFrame(venues)
df_search_data.columns = ['PostalCode', 'Borough', 'Neighbourhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']


In [140]:
# Build the one hot encoding for the discovered Foursquare data
tor_1H = pd.get_dummies(df_search_data[['VenueCategory']], prefix="", prefix_sep="")

tor_1H['PostalCode'] = df_search_data['PostalCode'] 
tor_1H['Borough'] = df_search_data['Borough'] 
tor_1H['Neighbourhoods'] = df_search_data['Neighbourhood'] 

fixed_columns = list(tor_1H.columns[-3:]) + list(tor_1H.columns[:-3])
tor_1H = tor_1H[fixed_columns]

In [139]:
#Build a dataframe for the One hot data grouped by relevent columns
tor_grouped = tor_1H.groupby(["PostalCode", "Borough", "Neighbourhoods"]).mean().reset_index()

In [138]:
#Disciver the top 10 venues per neighbourhood

num_top_venues = 10
indicators = ['st', 'nd', 'rd']


areaColumns = ['PostalCode', 'Borough', 'Neighbourhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = tor_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = tor_grouped['Borough']
neighborhoods_venues_sorted['Neighbourhoods'] = tor_grouped['Neighbourhoods']

for ind in np.arange(tor_grouped.shape[0]):
    row_categories = tor_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]


In [136]:
# Perform clusters analysis
kclusters = 5

tor_clustering = tor_grouped.drop(["PostalCode", "Borough", "Neighbourhoods"], 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_clustering)

In [135]:
# create a new dataframe that includes the cluster, top 10 and geo data
tor_merged = df_tor_only.copy()
tor_merged["Cluster Labels"] = kmeans.labels_
tor_merged = tor_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighbourhoods"], 1).set_index("PostalCode"), on="PostalCode")

In [145]:
# sort by Cluster Labels
tor_merged.sort_values(["Cluster Labels"], inplace=True)

(39, 16)


In [112]:
# map out the results using diff colours for each cluster

map_clusters = folium.Map(location=[43.653226, -79.383184], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['PostalCode'], tor_merged['Borough'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Display the most common Venue type per Cluster

In [146]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0 , ['Borough','1st Most Common Venue']]

,Borough,1st Most Common Venue
0,East Toronto,Pub
2,East Toronto,Indian Restaurant
3,East Toronto,Bar
5,Central Toronto,Coffee Shop
6,Central Toronto,Coffee Shop
7,Central Toronto,Italian Restaurant
8,Central Toronto,Grocery Store
1,East Toronto,Greek Restaurant
17,Downtown Toronto,Coffee Shop
11,Downtown Toronto,Coffee Shop


In [128]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1 , ['Borough','1st Most Common Venue']]

,Borough,1st Most Common Venue
10,Downtown Toronto,Park


In [129]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3 , ['Borough','1st Most Common Venue']]

,Borough,1st Most Common Venue
22,Central Toronto,Playground


In [133]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4 , ['Borough','1st Most Common Venue']]

,Borough,1st Most Common Venue
23,Central Toronto,Gym / Fitness Center
